In [1]:
from numpy import *
from scipy.sparse import *
import numpy as np
from sklearn.preprocessing import *

In [2]:
URM_all=load_npz("all_playlist_with_tracks_URM.npz")
trackIDs=load("fucking_ordered_tracks.npy")
playlistIDs=load("fucking_ordered_playlist.npy")
trackIDs.size,playlistIDs.size,URM_all

(100000, 45649, <45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 1040522 stored elements in Compressed Sparse Column format>)

In [3]:
#pBuilder=array([]).astype(int32)
#for i,p in enumerate(URM_all):
    #pBuilder=np.hstack((pBuilder,array([i]*p.size)))

In [4]:
#save("pBuilder.npy",pBuilder)
pBuilder=load("pBuilder.npy")
pBuilder.size

1040522

In [5]:
#tBuilder=array([]).astype(int32)
#for p in URM_all:
    #tBuilder=np.hstack((tBuilder,where(p.todense()!=0)[1]))

In [6]:
#save("tBuilder.npy",tBuilder)
tBuilder=load("tBuilder.npy")
tBuilder.size

1040522

In [7]:
rBuilder=np.ones(1040522).astype(int32)

In [8]:
class RandomRecommender(object):

    def fit(self, URM_train):
           
        self.numItems = URM_train.shape[1]
    
    
    def recommend(self, playlist_id, at=5):
        
        #genera 5 numeri da 0 a 99999
        recommended_index = np.random.choice(self.numItems, at)
        #prende le canzoni in quella posizione
        recommended_items = trackIDs[recommended_index]

        return recommended_items

In [9]:
class Top5Recommender(object):

    def recommend(self, playlist_id, at=5):
        
        recommended_items = array([1563309, 1363985, 3705881, 1595978, 3166665])

        return recommended_items

In [10]:
trackArtist=load_npz("fucking_ordered_track_artist.npz")
trackAlbum=load_npz("fucking_ordered_track_album.npz")
trackTag=load_npz("all_tracks_tags_TF_IDF.npz")
targetTrack=unique(genfromtxt("target_tracks.csv",dtype=int32,skip_header=1))
maskT=list(map(lambda x:x in targetTrack,trackIDs))

In [11]:
trackTag=normalize(trackTag)

In [13]:
Z=csc_matrix(ones((4,4)))
Z[:,0]=0
Z.eliminate_zeros()
Z.todense()

matrix([[ 0.,  1.,  1.,  1.],
        [ 0.,  1.,  1.,  1.],
        [ 0.,  1.,  1.,  1.],
        [ 0.,  1.,  1.,  1.]])

In [ ]:
"""
for i,ind in enumerate(where(array(maskT)==False)[0]):
    simByPlaylist[:,ind]=0
    if(i%1000==0):
        print(i)
"""        

In [14]:
Z=csr_matrix(ones((4,4)))
testMask=array([True,False,False,True])
Z[:,testMask]=0
Z.eliminate_zeros()
Z.todense()

matrix([[ 0.,  1.,  1.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 0.,  1.,  1.,  0.]])

In [15]:
csc_matrix(zeros((45,100)))*csc_matrix(zeros((100,30)))

<45x30 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>

In [27]:
class ArtistAlbumRecommender(object):

    def fit(self, URM_train, artistWeight, albumWeight, tagWeight):
        
        simByArtist=trackArtist*trackArtist.T
        simByAlbum=trackAlbum*trackAlbum.T
        simByPlaylist=URM_all.T*URM_all
        if(tagWeight!=0):
            simByTag=trackTag*trackTag.T
            simByTag.setdiag(0)
        else:
            simByTag=0
        simByPlaylist.setdiag(0)
        simByPlaylist=normalize(simByPlaylist)
        simByArtist.setdiag(0)
        simByAlbum.setdiag(0)
        S=artistWeight*simByArtist+albumWeight*simByAlbum+tagWeight*simByTag#+simByPlaylist#100kx100k
        S=S[:,array(maskT)]#100kx32k
        print("S="+str(S.shape))
        RR=URM_train*S#45kx32k
        print("RR="+str(RR.shape))
        myMax=max([max(x.data) for x in RR if x.data.size!=0])
        self.RR=RR-URM_train[:,array(maskT)]*myMax
        print("fittato")
        
    def recommend(self, i, at=5):
        
        sims=self.RR[i]
        recommended_items = trackIDs[flip(argsort(sims.todense()),1).take(range(at))]
        
        return recommended_items

In [19]:
def precision(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

In [20]:
def recall(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

In [21]:
def MAP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

In [22]:
def evaluate_algorithm(URM_test, recommender_object, at=5):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0
    
    num_eval = 0

    oldprogress=0
    for i,playlist_id in enumerate(playlistIDs):
        
        #va alla riga i-esima, che contiene la playlist attuale
        relevant_index = URM_test[i].indices
        #prende le track della playlist i-esima
        relevant_items = trackIDs[relevant_index]
        if len(relevant_items)>0:
            # HO MODIFICATO CON i
            #recommended_items = recommender_object.recommend(playlist_id, at=at)
            recommended_items = recommender_object.recommend(i, at=at)
            #print(str(playlist_id)+" "+str(recommended_items))
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_MAP += MAP(recommended_items, relevant_items)
        progress=i/45649*100
        if(progress-oldprogress>1):
            print("\r{:.2f}%".format(progress), end="")
            oldprogress=progress
        

    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval
    
    print("\r100%", end="")
    print(" Precision = {:.6f}, Recall = {:.6f}, MAP = {:.6f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP)) 


In [23]:
#ATTENZIONE, controllare che entrambe le matrici alla fine siano 45649x100000
# sennò si spacca tutto
train_test_split = 0.80
numInteractions = URM_all.nnz
train_mask = np.random.choice([True,False], numInteractions, p=[train_test_split, 1-train_test_split])

URM_train=(coo_matrix((rBuilder[train_mask],(pBuilder[train_mask],tBuilder[train_mask]))))
URM_train=URM_train.tocsr()

test_mask=logical_not(train_mask)
URM_test=(coo_matrix((rBuilder[test_mask],(pBuilder[test_mask],tBuilder[test_mask]))))
URM_test=URM_test.tocsr()
URM_train,URM_test

(<45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 832613 stored elements in Compressed Sparse Row format>,
 <45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 207909 stored elements in Compressed Sparse Row format>)

In [24]:
randomRecommender = RandomRecommender()
randomRecommender.fit(URM_train)
evaluate_algorithm(URM_test, randomRecommender)
#Precision = 0.000085, Recall = 0.000072, MAP = 0.000058

100% Precision = 0.000097, Recall = 0.000062, MAP = 0.000057


In [25]:
top5recommender = Top5Recommender()
evaluate_algorithm(URM_test,top5recommender)
#Precision = 0.002344, Recall = 0.001981, MAP = 0.001480

100% Precision = 0.002450, Recall = 0.001973, MAP = 0.001540


In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,0.5,0)
evaluate_algorithm(URM_test, artistAlbumRecommender)

In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,1,0)
evaluate_algorithm(URM_test, artistAlbumRecommender)

In [ ]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,1.5,0)
evaluate_algorithm(URM_test, artistAlbumRecommender)
#Precision = 0.040979, Recall = 0.047600, MAP = 0.040184

In [28]:
artistAlbumRecommender = ArtistAlbumRecommender()
artistAlbumRecommender.fit(URM_train,1,1.5,0)
evaluate_algorithm(URM_test, artistAlbumRecommender)

/home/oppy/.local/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


S=(100000, 32195)
RR=(45649, 32195)
fittato
100% Precision = 0.000074, Recall = 0.000105, MAP = 0.000057
